# Calibration Tutorial - Crane, OR - Irrigated Flux Plot

## Step 2: Calibrate with PEST++

Now we see if we can improve the model's performance through calibration.

### PEST++ Resources

PEST++ has excellent documentation covering both theory and practice:

1. **The PEST Manual 4th Ed.** (Doherty, 2002): https://www.epa.gov/sites/default/files/documents/PESTMAN.PDF
2. **GMDSI tutorial notebooks**: https://github.com/gmdsi/GMDSI_notebooks
3. **PEST++ User's Manual**: https://github.com/usgs/pestpp/blob/master/documentation/pestpp_users_manual.md
4. **PEST Book** (Doherty, 2015): https://pesthomepage.org/pest-book

**Note:** We use SSEBop ETf and SNODAS SWE for calibration - these are widely available remote sensing products.

### PEST++ Installation

Install PEST++ via conda-forge (recommended):

```bash
conda install conda-forge::pestpp
```

This installs `pestpp-ies` and other PEST++ tools directly into your conda environment. Verify the installation:

```bash
pestpp-ies --version
```

**Alternative:** For manual installation from source, see the [PEST++ GitHub releases](https://github.com/usgs/pestpp/releases).

In [1]:
import os
import sys

root = os.path.abspath("../..")
sys.path.append(root)

from swimrs.calibrate.pest_builder import PestBuilder
from swimrs.calibrate.run_pest import run_pst
from swimrs.container import SwimContainer
from swimrs.swim.config import ProjectConfig

## 1. Load Configuration

Load the project configuration and prepare paths.

In [2]:
project = "3_Crane"
project_ws = os.path.abspath(".")

config_path = os.path.join(project_ws, "3_Crane.toml")

config = ProjectConfig()
config.read_config(config_path, project_ws)

## 2. Generate Observation Files

PEST++ needs observation files (ETf and SWE) to compare against model predictions. We export these from the SwimContainer built in the data preparation step.

The export creates per-field numpy files:
- `obs_etf_{field_id}.np`: ETf observations with irrigation mask switching
- `obs_swe_{field_id}.np`: SWE observations from SNODAS

In [3]:
# Open container for observations export and PEST++ setup
container_path = os.path.join(project_ws, "data", f"{project}.swim")
container = SwimContainer.open(container_path, mode="r")

# Export observation files for PEST++ calibration
# Note: obs_folder is set to {pest_run_dir}/obs in the TOML config
obs_dir = os.path.join(project_ws, "data", "pestrun", "obs")
os.makedirs(obs_dir, exist_ok=True)

container.export.observations(
    output_dir=obs_dir,
    etf_model="ssebop",
    masks=("irr", "inv_irr"),
    irr_threshold=0.1,
)
print(f"Observation files written to {obs_dir}")

Observation files written to /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/obs


## 3. Build PEST++ Control Files

The `PestBuilder` class sets up everything needed for PEST++ calibration:

### Calibration Loop
1. Initialize model with initial parameter values
2. Run model, write results
3. Compare results to observations (ETf, SWE)
4. Propose new parameters
5. Repeat until convergence

### Tunable Parameters
SWIM uses 8 tunable parameters:
- **Soil water:** `aw` (available water), `rew` (readily evaporable water), `tew` (total evaporable water)
- **NDVI-Kcb relationship:** `ndvi_alpha`, `ndvi_beta`
- **Stress threshold:** `mad` (management allowable depletion)
- **Snow melt:** `swe_alpha`, `swe_beta`

In [4]:
# PestBuilder auto-generates custom_forward_run.py using the process package
# with portable swim_input.h5 file for fully self-contained workers
builder = PestBuilder(config, container=container, use_existing=False, verbose=False)

### Build the .pst Control File

The `build_pest()` method:
- Copies project files to a `pest/` directory
- Creates the `.pst` control file
- Sets up parameter templates (`.tpl`) and instruction files (`.ins`)

**Note:** ETf observations are sparse (only on satellite capture dates). The PEST builder automatically assigns weight 1.0 to valid observations and weight 0.0 to missing dates, ensuring we only calibrate against actual satellite captures.

In [5]:
# Build the pest control file
# WARNING: This will erase any existing pest directory!
builder.build_pest(target_etf=config.etf_target_model, members=config.etf_ensemble_members)

noptmax:0, npar_adj:8, nnz_obs:1754


noptmax:0, npar_adj:8, nnz_obs:1754


In [6]:
# Show the files created in the pest directory
pest_files = [
    f
    for f in sorted(os.listdir(builder.pest_dir))
    if os.path.isfile(os.path.join(builder.pest_dir, f))
]
print("Files in pest directory:")
for f in pest_files:
    print(f"  {f}")

Files in pest directory:
  3_Crane.idx.csv
  3_Crane.pst
  3_crane.insfile_data.csv
  3_crane.obs_data.csv
  3_crane.par_data.csv
  3_crane.pargp_data.csv
  3_crane.tplfile_data.csv
  custom_forward_run.py
  etf_S2.ins
  mult2model_info.csv
  p_aw_S2_0_constant.csv.tpl
  p_kr_alpha_S2_0_constant.csv.tpl
  p_ks_alpha_S2_0_constant.csv.tpl
  p_mad_S2_0_constant.csv.tpl
  p_ndvi_0_S2_0_constant.csv.tpl
  p_ndvi_k_S2_0_constant.csv.tpl
  p_swe_alpha_S2_0_constant.csv.tpl
  p_swe_beta_S2_0_constant.csv.tpl
  params.csv
  swe_S2.ins
  swim_input.h5


### Examine the .pst File

The PEST++ version 2 control file is concise, delegating details to external files:

In [7]:
with open(builder.pst_file) as f:
    print(f.read())

pcf version=2
* control data keyword
pestmode                                 estimation
noptmax                                 0
svdmode                                 1
maxsing                          10000000
eigthresh                           1e-06
eigwrite                                1
* parameter groups external
3_crane.pargp_data.csv
* parameter data external
3_crane.par_data.csv
* observation data external
3_crane.obs_data.csv
* model command line
python custom_forward_run.py
* model input external
3_crane.tplfile_data.csv
* model output external
3_crane.insfile_data.csv



## 4. Configure and Test

Now we:
1. Build the localizer matrix (links parameters to relevant observations)
2. Run spinup to save initial water balance state
3. Do a dry run to verify everything works
4. Set control parameters for the full calibration

In [8]:
# Build localizer matrix
# - SWE observations update only swe_alpha and swe_beta
# - ETf observations update all other parameters
builder.build_localizer()

# Run spinup to save water balance state as initial conditions for calibration runs
# This runs the model once and saves the final state to spinup.json
print("Running spinup...")
builder.spinup(overwrite=True)

# Run a minimal model run to verify the setup
print("\nRunning dry run...")
builder.dry_run()

# Configure for 3 optimization iterations with 20 realizations each
# Increase realizations (e.g., 100-200) for production runs
builder.write_control_settings(noptmax=3, reals=20)

noptmax:0, npar_adj:8, nnz_obs:1754
Running spinup...



Running dry run...


noptmax:3, npar_adj:8, nnz_obs:1754


In [9]:
# Show the updated control file with calibration settings
with open(builder.pst_file) as f:
    print(f.read())

pcf version=2
* control data keyword
pestmode                                 estimation
noptmax                                 3
svdmode                                 1
maxsing                          10000000
eigthresh                           1e-06
eigwrite                                1
ies_localizer                  loc.mat
ies_num_reals                  20
ies_drop_conflicts             true
* parameter groups external
3_crane.pargp_data.csv
* parameter data external
3_crane.par_data.csv
* observation data external
3_crane.obs_data.csv
* model command line
python custom_forward_run.py
* model input external
3_crane.tplfile_data.csv
* model output external
3_crane.insfile_data.csv



## 5. Run PEST++ Calibration

Now we launch PEST++ with parallel workers. Adjust `workers` to match your machine's capabilities.

**Important:** This can take a while. Watch for the progress indicator.

In [10]:
workers = config.workers or 6
_pst = f"{config.project_name}.pst"

print(f"Starting PEST++ with {workers} workers...")
print(f"Control file: {_pst}")

run_pst(
    builder.pest_dir,
    "pestpp-ies",
    _pst,
    num_workers=workers,
    worker_root=builder.workers_dir,
    master_dir=builder.master_dir,
    cleanup=True,
    verbose=True,
)

Starting PEST++ with 6 workers...
Control file: 3_Crane.pst
master:pestpp-ies 3_Crane.pst /h :5005 in /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/master


             pestpp-ies: a GLM iterative ensemble smoother

                   by the PEST++ development team

...processing command line: ' pestpp-ies 3_Crane.pst /h :5005'
...using panther run manager in master mode using port 5005


version: 5.2.25
binary compiled on Jan 13 2026 at 20:33:04
using control file: "3_Crane.pst"
in directory: "/home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/master"
on host: "fcfc-mcozoran"
on a(n) linux operating system
with release configuration
started at 01/28/26 15:59:03

processing control file 3_Crane.pst




:~-._                                                 _.-~:
: :.~^o._        ________---------________        _.o^~.:.:
 : ::.`?88booo~~~.::::::::...::::::::::::..~~oood88P'.::.:
 :  ::: `?88P .:::....         ........:::::. ?88P' :::. :
  :  :::. `? .::.            . ...........:::. P' .:::. :
   :  :::   ... ..  ...       .. .::::......::.   :::. :
   `  :' .... ..  .:::::.     . ..:::::::....:::.  `: .'
    :..    ____:::::::::.  . . ....:::::::::____  ... :
   :... `:~    ^~-:::::..  .........:::::-~^    ~::.::::
   `.::. `\   (8)  \b:::..::.:.:::::::d/  (8)   /'.::::'
    ::::.  ~-._v    |b.::::::::::::::d|    v_.-~..:::::
    `.:::::... ~~^?888b..:::::::::::d888P^~...::::::::'
     `.::::::::::....~~~ .:::::::::~~~:::::::::::::::'
      `..:::::::::::   .   ....::::    ::::::::::::,'
        `. .:::::::    .      .::::.    ::::::::'.'
          `._ .:::    .        :::::.    :::::_.'
             `-. :    .        :::::      :,-'
                :.   :___     .:::___   .::
    

worker:pestpp-ies 3_Crane.pst /h localhost:5005 in /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/workers/worker_2
worker:pestpp-ies 3_Crane.pst /h localhost:5005 in /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/workers/worker_3
worker:pestpp-ies 3_Crane.pst /h localhost:5005 in /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/workers/worker_4
worker:pestpp-ies 3_Crane.pst /h localhost:5005 in /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/workers/worker_5
...obscov loaded  from observation weights
...drawing parameter realizations:  20
...not using prior parameter covariance matrix scaling
...drawing observation noise realizations:  20
observations not grouped by observations groups, reordering obs ensemble


...setting weights ensemble from control file weights
...saved weight ensemble to  3_Crane.weights.csv
...adding 'base' parameter values to ensemble
...adding 'base' observation values to ensemble
...adding 'base' weight values to weight ensemble
...saved initial parameter ensemble to  3_Crane.0.par.csv


...saved obs+noise observation ensemble (obsval + noise realizations) to  3_Crane.obs+noise.csv
...using subset in lambda testing, percentage of realizations used in subset testing:  10
...subset how:  RANDOM
...centering on ensemble mean vector
...running initial ensemble of size 20
    running model 20 times
    starting at 01/28/26 15:59:05

    waiting for agents to appear...


PANTHER progress
   avg = average model run time in minutes
   runs(C = completed | F = failed | T = timed out)
   agents(R = running | W = waiting | U = unavailable)
--------------------------------------------------------------------------------


...

01/28 16:00:57 remaining file transfers: 0

   16 runs complete :  0 runs failed
   0.0535 avg run time (min) : 0.162 run mgr time (min)
   6 agents connected




...phi summary for entire ensemble using lambda,scale_fac 5.625e-05 , 1.1 , 
       phi type           mean            std            min            max
       measured        261.978        32.3876        145.923        301.119
         actual        145.734        1.24304        143.405        148.145
     note: 'measured' phi reported above includes 
           realizations of measurement noise, 
           'actual' phi does not.
...last best mean phi * acceptable phi factor:  275.874
...current best mean phi:  261.978

  ---  updating parameter ensemble  ---  

  ---  updating lambda to  4.21875e-05  ---  

  ---  EnsembleMethod iteration 3 report  ---  
   number of active realizations:   20
   number of model runs:            176


      current obs ensemble saved to 3_Crane.3.obs.csv
      current par ensemble saved to 3_Crane.3.par.csv


saved par and rei files for realization BASE for iteration 3


saved par and rei files for realization BASE
       phi type           mean            std            min            max
       measured        261.978        32.3876        145.923        301.119
         actual        145.734        1.24304        143.405        148.145
     note: 'measured' phi reported above includes 
           realizations of measurement noise, 
           'actual' phi does not.
  ---  observation group phi summary ---  
       (computed using 'actual' phi)
           (sorted by mean phi)
group                               count  nconflict      mean       std       min       max   percent       std
oname:obs/obs_etf_s2.np_otype:arr     504         76       145      1.15       143       147      99.4     0.169
oname:obs/obs_swe_s2.np_otype:arr    1175        276      0.82     0.251     0.556      1.39     0.562     0.169


   ---  parameter group change summary  ---    
group     count  mean chg  std chg  n at ubnd  % at ubnd  n at lbnd  % at lbnd  n std decr
mad

## 6. Check Results

After successful calibration, parameter files are saved for each optimization iteration:

In [11]:
# Parameter files are in master_dir after calibration
import os
import shutil

results_dir = builder.master_dir if os.path.exists(builder.master_dir) else builder.pest_dir

par_files = [f for f in sorted(os.listdir(results_dir)) if ".par.csv" in f]
print(f"Parameter files (in {results_dir}):")
for f in par_files:
    print(f"  {f}")

if par_files:
    final_par = par_files[-1]
    print(f"\nFinal calibrated parameters: {final_par}")

    # Copy final parameters to pest/archive/ for consistency with calibration.py script
    archive_dir = os.path.join(builder.pest_dir, "archive")
    os.makedirs(archive_dir, exist_ok=True)
    src = os.path.join(results_dir, final_par)
    dst = os.path.join(archive_dir, final_par)
    shutil.copy2(src, dst)
    print(f"Archived to: {dst}")

# Close the container
container.close()
print("\nContainer closed.")

Parameter files (in /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/master):
  3_Crane.0.par.csv
  3_Crane.1.par.csv
  3_Crane.2.par.csv
  3_Crane.3.par.csv

Final calibrated parameters: 3_Crane.3.par.csv
Archived to: /home/dgketchum/code/swim-rs/examples/3_Crane/data/pestrun/pest/archive/3_Crane.3.par.csv

Container closed.


## Summary

You've set up and run a PEST++ calibration using:
- SSEBop ETf observations from Landsat
- SNODAS SWE observations
- Iterative Ensemble Smoother (pestpp-ies)

The parameter files (e.g., `3_Crane.3.par.csv`) contain the calibrated parameter sets.

**Next step:** In notebook `03_calibrated_model.ipynb`, we'll run the model in forecast mode with calibrated parameters and evaluate the improvement.